In [4]:
import sys
from pathlib import Path

parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))

In [5]:
import numpy as np
import cv2
import dlib
import math
from data_processing.classes.image_processing import ImageProcessor
from data_processing.classes.blink_detector import BlinkDetector 
global_detector = dlib.get_frontal_face_detector()
global_predictor = dlib.shape_predictor('../shape_predictor_68_face_landmarks.dat')
global_sr_model = cv2.dnn_superres.DnnSuperResImpl_create()
global_sr_model.readModel("../EDSR_x4.pb")
global_sr_model.setModel("edsr", 2)
ImageProcessor = ImageProcessor(global_detector, global_predictor, global_sr_model)

In [6]:

def get_combined_eyes(frame, global_detector, global_predictor, target_size=(200, 100)):
    """
    Detects, enhances, and combines the eye regions including the nose bridge from the frame.
    Args:
        frame: The input image frame.
        global_detector: Face detector.
        global_predictor: Landmark predictor.
        target_size: Target size for resizing the combined eye region.
    Returns:
        The combined eye regions including the nose bridge, or None if not detected.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = global_detector(gray)

    # super resolution image
    for face in faces:
        landmarks = global_predictor(gray, face)

        forehead_points = [20, 21, 22, 23, 0 ,16]
        left_eye_landmarks = [36, 37, 38, 39, 40, 41]
        right_eye_landmarks = [42, 43, 44, 45, 46, 47]
        nose_bridge_points = [27, 28, 29]

        blink_detctor = BlinkDetector()
        
        if blink_detctor.detect_blink(landmarks, left_eye_landmarks, right_eye_landmarks):
            print("Blink Detected")

        combined_eye_region, _ = ImageProcessor.extract_eye_region(
            frame, landmarks, left_eye_landmarks, right_eye_landmarks, nose_bridge_points, forehead_points)

        if isinstance(combined_eye_region, np.ndarray):

            # Apply super-resolution
            # combined_eye_super_res = ImageProcessor.enhance_image_resolution(combined_eye_region, global_sr_model)

            # Resize to the final target size
            combined_eye_final_resized = cv2.resize(combined_eye_region, target_size, interpolation=cv2.INTER_AREA)

            # combined_eye_final_resized = cv2.cvtColor(combined_eye_final_resized, cv2.COLOR_BGR2GRAY)
            combined_eye_final_resized = combined_eye_final_resized.astype(np.float32) / 255.0

            return combined_eye_final_resized

    return None

In [8]:
image = cv2.imread("../data/eloise/calibration_images/eloise_56753efe-f44b-4043-8d5c-481218c2bc8f.png")
_ = get_combined_eyes(image, global_detector, global_predictor) 

Blink Detected
Blink Detected


In [7]:
image = cv2.imread("../data/William/calibration_images/William_003ae256-4461-4ca5-824a-bb59cf6c73c0.png")
_ = get_combined_eyes(image, global_detector, global_predictor)

In [8]:
#test all images in Will
import os


# Load all images
for subdir, dirs, files in os.walk("../data/Will/calibration_images"):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".png"):
            print(filepath)
            image = cv2.imread(filepath)
            _ = get_combined_eyes(image, global_detector, global_predictor) 

../data/Will/calibration_images/Will_51a13692-8bbc-4a74-9c26-7d23f2c3763b.png
../data/Will/calibration_images/Will_29d6e526-2eda-44b1-907e-510c968f50a8.png
../data/Will/calibration_images/Will_d3dccad0-800f-4de8-bcad-24835b964a58.png
../data/Will/calibration_images/Will_a088e932-4916-4484-ae5e-74fd8f18e44a.png
../data/Will/calibration_images/Will_e390b8e6-9295-4e48-8d44-d2aeb84f5038.png
../data/Will/calibration_images/Will_64286508-4a2b-4544-ac0a-aea27cee7dcc.png
../data/Will/calibration_images/Will_24cb5f99-3442-4557-8f84-5cf6afc48384.png
../data/Will/calibration_images/Will_bdffbf0d-ef10-4192-aa08-c0574cf6c299.png
../data/Will/calibration_images/Will_0831b823-4eb5-402b-a450-61849cd70b94.png
../data/Will/calibration_images/Will_90ae1720-a648-41c5-bbf4-a763982cbe09.png
../data/Will/calibration_images/Will_0dd10e31-b4c0-46e4-aae5-9401259105ad.png
../data/Will/calibration_images/Will_58f897a5-7eaa-4c55-9fbc-aa8a758b8465.png
../data/Will/calibration_images/Will_bd3647a3-3b9a-4fc0-b627-445